In [3]:
#require "core";;
#require "yaml";;
#require "fpath"
#require "yaml.unix"
#require "lwt"
#require "cohttp"
#require "cohttp-lwt-unix"
#require "yojson"
#require "ppx_yojson_conv"
#use "/home/opam/work/lib/util.ml"


val to_string :
  [> `String of 'a ] -> ('a, [> `Msg of string ]) Rresult.result = <fun>
val to_bool : [> `Bool of 'a ] -> ('a, [> `Msg of string ]) Rresult.result =
  <fun>
val to_float : [> `Float of 'a ] -> ('a, [> `Msg of string ]) Rresult.result =
  <fun>
val map :
  ('a -> 'b) ->
  [> `A of 'a list ] ->
  ([> `A of 'b list ], [> `Msg of string ]) Rresult.result = <fun>


In [13]:
open Lwt
open Cohttp
open Cohttp_lwt_unix


let keyvallist_to_form_urlencoded p =
  p |> List.map (fun (k,v) -> Printf.sprintf "%s=%s" k v)
  |> String.concat "&"
  |> Cohttp_lwt.Body.of_string

let keyvallist_to_form_json p = p
  |> List.map (fun (k,v) -> Printf.sprintf {|"%s": "%s"|} k v)
  |> String.concat ", "
  |> Printf.sprintf "{%s}"
  |> Cohttp_lwt.Body.of_string

let (alpaca_secret, alpaca_key) = 
  let yaml = Yaml_unix.of_file_exn (Fpath.v "keys.paper.yaml") in
  let key = yaml |> get_value "key" |> get_string_opt |> get_string in 
  let secret = yaml |> get_value "secret" |> get_string_opt |> get_string in
  (secret, key)

let headers = Header.init ()
  |> fun h -> Header.add_list h 
              [("APCA-API-KEY-ID", alpaca_key); 
               ("APCA-API-SECRET-KEY", alpaca_secret)]

type order = {
  symbol: string;
  qty: int;
  side: string;
  typexx: string;
  time_in_force: string;
} [@@deriving yojson]

let order = {
  symbol= "MSFT";
  qty=10;
  side="buy";
  typexx="market";
  time_in_force="day";
}

let stripchars s cs =
  let len = String.length s in
  let res = Bytes.create len in
  let rec aux i j =
    if i >= len
    then Bytes.to_string (Bytes.sub res 0 j)
    else if String.contains cs s.[i] then
      aux (succ i) (j)
    else begin
      Bytes.set res j s.[i];
      aux (succ i) (succ j)
    end
  in
  aux 0 0

let _body = yojson_of_order order |> Yojson.Safe.to_string
let body = Cohttp_lwt.Body.of_string (stripchars _body "xx")

let order_list = [("symbol", "MSFT"); ("qyt","10"); ("side","buy"); ("type","market"); ("time_in_force","day")]

let test_post =
  let uri = Uri.of_string "https://paper-api.alpaca.markets/v2/orders" in
    Cohttp_lwt_unix.Client.post ~headers ~body uri >>= fun (resp, body) ->
      let code = resp |> Response.status |> Code.code_of_status in
      Printf.printf "Response code: %d\n" code;
      Printf.printf "Headers: %s\n" (resp |> Response.headers |> Header.to_string);
      body |> Cohttp_lwt.Body.to_string >|= fun body ->
      Printf.printf "Body of length: %d\n" (String.length body);
      body

let () =
  let respBody = Lwt_main.run test_post in
  print_endline (respBody)

val keyvallist_to_form_urlencoded :
  (string * string) list -> Cohttp_lwt.Body.t = <fun>


val keyvallist_to_form_json : (string * string) list -> Cohttp_lwt.Body.t =
  <fun>


val alpaca_secret : string = "qEmsGIbj8WSBK3DLVDlwcB6KJ3JsLcndnuCHdLIu"
val alpaca_key : string = "PKLDSENB0U8ST5KZI612"


val headers : Cohttp.Header.t = <abstr>


type order = {
  symbol : string;
  qty : int;
  side : string;
  typexx : string;
  time_in_force : string;
}
val order_of_yojson : Ppx_yojson_conv_lib.Yojson.Safe.t -> order = <fun>
val yojson_of_order : order -> Ppx_yojson_conv_lib.Yojson.Safe.t = <fun>


...

val _body : string =
  "{\"symbol\":\"MSFT\",\"qty\":10,\"side\":\"buy\",\"typexx\":\"market\",\"time_in_force\":\"day\"}"


val body : Cohttp_lwt.Body.t =
  `String
    "{\"symbol\":\"MSFT\",\"qty\":10,\"side\":\"buy\",\"type\":\"market\",\"time_in_force\":\"day\"}"


val order_list : (string * string) list =
  [("symbol", "MSFT"); ("qyt", "10"); ("side", "buy"); ("type", "market");
   ("time_in_force", "day")]


val test_post : string Lwt.t = <abstr>


Response code: 200
Headers: access-control-allow-credentials: true
connection: keep-alive
content-length: 735
content-type: application/json; charset=UTF-8
date: Fri, 05 Feb 2021 17:30:19 GMT
server: nginx/1.16.1
vary: Origin


Body of length: 735
{"id":"eac88bba-0c80-4149-b174-d152fe307c2e","client_order_id":"656917ac-2860-47c8-b673-c27ee727ecc3","created_at":"2021-02-05T17:30:19.497139Z","updated_at":"2021-02-05T17:30:19.497139Z","submitted_at":"2021-02-05T17:30:19.483706Z","filled_at":null,"expired_at":null,"canceled_at":null,"failed_at":null,"replaced_at":null,"replaced_by":null,"replaces":null,"asset_id":"b6d1aa75-5c9c-4353-a305-9e2caa1925ab","symbol":"MSFT","asset_class":"us_equity","qty":"10","filled_qty":"0","filled_avg_price":null,"order_class":"","order_type":"market","type":"market","side":"buy","time_in_force":"day","limit_price":null,"stop_price":null,"status":"accepted","extended_hours":false,"legs":null,"trail_percent":null,"trail_price":null,"hwm":null}


In [ ]:
let body = keyvallist_to_form_json order_list 

In [ ]:
  let body = keyvallist_to_form_urlencoded order_list in

In [4]:
type order = {
  symbol: string;
  qty: int;
  side: string;
  type_trade: string;
  time_in_force: string;
} [@@deriving yojson]

type order = {
  symbol : string;
  qty : int;
  side : string;
  type_trade : string;
  time_in_force : string;
}
val order_of_yojson : Ppx_yojson_conv_lib.Yojson.Safe.t -> order = <fun>
val yojson_of_order : order -> Ppx_yojson_conv_lib.Yojson.Safe.t = <fun>


In [9]:
type order = {
  symbol: string;
  qty: int;
  side: string;
  type_: string;
  time_in_force: string;
} [@@deriving yojson]

let order = {
  symbol= "MSFT";
  qty=10;
  side="buy";
  type_="market";
  time_in_force="day";
}

let stripchars s cs =
  let len = String.length s in
  let res = Bytes.create len in
  let rec aux i j =
    if i >= len
    then Bytes.to_string (Bytes.sub res 0 j)
    else if String.contains cs s.[i] then
      aux (succ i) (j)
    else begin
      Bytes.set res j s.[i];
      aux (succ i) (succ j)
    end
  in
  aux 0 0

let _body = yojson_of_order order |> Yojson.Safe.to_string
let body = stripchars _body "_"

type order = {
  symbol : string;
  qty : int;
  side : string;
  type_ : string;
  time_in_force : string;
}
val order_of_yojson : Ppx_yojson_conv_lib.Yojson.Safe.t -> order = <fun>
val yojson_of_order : order -> Ppx_yojson_conv_lib.Yojson.Safe.t = <fun>


val order : order =
  {symbol = "MSFT"; qty = 10; side = "buy"; type_ = "market";
   time_in_force = "day"}


val stripchars : string -> string -> string = <fun>


val _body : string =
  "{\"symbol\":\"MSFT\",\"qty\":10,\"side\":\"buy\",\"type_\":\"market\",\"time_in_force\":\"day\"}"


val body : string =
  "{\"symbol\":\"MSFT\",\"qty\":10,\"side\":\"buy\",\"type\":\"market\",\"timeinforce\":\"day\"}"
